In [30]:
# Example 7-1. A decorator usually replaces a function with a different one.
def deco(func):
    def inner():
        print('running inner()')
    return inner

# codes 1 and 2 has the same effect

#1
target = deco(target)
print(target)
target()

#2
@deco
def target():
    print('running target()')
print(target)
target()

<function deco.<locals>.inner at 0x7fd8bc2a98c8>
running inner()
<function deco.<locals>.inner at 0x7fd8bc2a9ea0>
running inner()


In [38]:
# Example 7-2. The registration.py module
registry = []  # <1>

def register(func):  # <2>
    print('running register(%s)' % func)  # <3>
    registry.append(func)  # <4>
    return func  # <5>

@register  # <6>
def f1():
    print('running f1()')

@register
def f2():
    print('running f2()')

def f3():  # <7>
    print('running f3()')

def main():  # <8>
    print('\nrunning main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()

if __name__=='__main__':
    main()  # <9>

running register(<function f1 at 0x7fd8bc2a99d8>)
running register(<function f2 at 0x7fd8bc2cd598>)

running main()
registry -> [<function f1 at 0x7fd8bc2a99d8>, <function f2 at 0x7fd8bc2cd598>]
running f1()
running f2()
running f3()


In [25]:
# KOW teh same efekt as in Example 7-2
f2x = register(f2)
f2x()

running register(<function f2 at 0x7fd8bc300d90>)
running f2()


In [48]:
# Example 7-3. The promos list is filled by the promotion decorator.
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')


class LineItem:

    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity


class Order:  # the Context

    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

# BEGIN STRATEGY_BEST4

promos = []  # <1>

def promotion(promo_func):  # <2>
    promos.append(promo_func)
    return promo_func

@promotion  # <3>
def fidelity(order):
    """5% discount for customers with 1000 or more fidelity points"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item(order):
    """10% discount for each LineItem with 20 or more units"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

@promotion
def large_order(order):
    """7% discount for orders with 10 or more distinct items"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

def best_promo(order):  # <4>
    """Select best discount available
    """
    return max(promo(order) for promo in promos)

print("promos:")
print(promos)
# BEGIN STRATEGY_BEST_TESTS
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, .5), LineItem('apple', 10, 1.5), LineItem('watermellon', 5, 5.0)]
banana_cart = [LineItem('banana', 30, .5), LineItem('apple', 10, 1.5)]
long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]

print("Order(joe, long_order, best_promo) : ",Order(joe, long_order, best_promo))
print("Order(joe, banana_cart, best_promo) : ", Order(joe, banana_cart, best_promo))
print("Order(ann, cart, best_promo) : ", Order(ann, cart, best_promo))

promos:
[<function fidelity at 0x7fd8bc28c840>, <function bulk_item at 0x7fd8bc2a9158>, <function large_order at 0x7fd8bc300950>]
Order(joe, long_order, best_promo) :  <Order total: 10.00 due: 9.30>
Order(joe, banana_cart, best_promo) :  <Order total: 30.00 due: 28.50>
Order(ann, cart, best_promo) :  <Order total: 42.00 due: 39.90>


In [49]:
# Example 7-5. Variable b is local, because it is assigned a value in the body of the function
b = 6
def f2(a):
    print(a)
    print(b)
    b = 9
f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

In [56]:
# Example 7-5. - CORRECT
b = 6
def f3(a):
    global b
    print(a)
    print(b)
    b = 9
f3(3)
b

3
6


9

In [58]:
# Example 7-6. Disassembly of the f3 function
from dis import dis
dis(f3)

  5           0 LOAD_GLOBAL              0 (print)
              3 LOAD_FAST                0 (a)
              6 CALL_FUNCTION            1 (1 positional, 0 keyword pair)
              9 POP_TOP

  6          10 LOAD_GLOBAL              0 (print)
             13 LOAD_GLOBAL              1 (b)
             16 CALL_FUNCTION            1 (1 positional, 0 keyword pair)
             19 POP_TOP

  7          20 LOAD_CONST               1 (9)
             23 STORE_GLOBAL             1 (b)
             26 LOAD_CONST               0 (None)
             29 RETURN_VALUE


In [60]:
# Example 7-8. average_oo.py: A class to calculate a running average.
class Averager():
    def __init__(self):
        self.series = []
    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)

avg = Averager()
print("avg(10) : ", avg(10))
print("avg(11) : ", avg(11))
print("avg(12) : ", avg(12))

avg(10) :  10.0
avg(11) :  10.5
avg(12) :  11.0


In [61]:
# Example 7-9. average.py: a higher-order function to calculate a running average.
def make_averager():
    series = []
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)
    return averager

avg = make_averager()
print("avg(10) : ", avg(10))
print("avg(11) : ", avg(11))
print("avg(12) : ", avg(12))

avg(10) :  10.0
avg(11) :  10.5
avg(12) :  11.0


In [68]:
# Example 7-11. / 7-12. Inspecting the function created by make_averager in Example 7-9
print("avg.__code__.co_varnames : ", avg.__code__.co_varnames)
print("avg.__code__.co_freevars : ", avg.__code__.co_freevars)
print("avg.__closure__ : ", avg.__closure__)
print("avg.__closure__[0].cell_contents : ", avg.__closure__[0].cell_contents)

avg.__code__.co_varnames :  ('new_value', 'total')
avg.__code__.co_freevars :  ('series',)
avg.__closure__ :  (<cell at 0x7fd8bc3d0eb8: list object at 0x7fd8bc25ee08>,)
avg.__closure__[0].cell_contents :  [10, 11, 12]


In [1]:
# Example 7-13. A broken higher-order function to calculate a running average without keeping all history.
def make_averager():
    count = 0
    total = 0
    def averager(new_value):
        count += 1
        total += new_value
        return total / count
    return averager
avg = make_averager()
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

In [3]:
# Example 7-14. Calculate a running average without keeping all history. Fixed with the use of nonlocal
def make_averager():
    count = 0
    total = 0
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count
    return averager
avg = make_averager()
avg(10)
avg(12)

11.0

In [14]:
# Example 7-15. A simple decorator to output the running time of functions.
import time
def clock(func):
    def clocked(*args): #
        t0 = time.perf_counter()
        result = func(*args) #
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked #

@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

if __name__=='__main__':
    print('*' * 40, 'Calling snooze(.123)')
    snooze(.123)
    print('*' * 40, 'Calling factorial(6)')
    print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12467023s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000082s] factorial(1) -> 1
[0.00005883s] factorial(2) -> 2
[0.00011025s] factorial(3) -> 6
[0.00015951s] factorial(4) -> 24
[0.00020916s] factorial(5) -> 120
[0.00026023s] factorial(6) -> 720
6! = 720


In [16]:
# The same code effect OR NOT ???
@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)
factorial = clock(factorial)
print('*' * 40, 'Calling factorial(6)')
print('6! =', factorial(6))
print()
print("!!! factorial.__name__ : ", factorial.__name__, "!!!")

**************************************** Calling factorial(6)
[0.00000087s] factorial(1) -> 1
[0.00013742s] clocked(1) -> 1
[0.00018062s] factorial(2) -> 2
[0.00021608s] clocked(2) -> 2
[0.00025114s] factorial(3) -> 6
[0.00031661s] clocked(3) -> 6
[0.00038163s] factorial(4) -> 24
[0.00047531s] clocked(4) -> 24
[0.00054289s] factorial(5) -> 120
[0.00060750s] clocked(5) -> 120
[0.00067439s] factorial(6) -> 720
[0.00074001s] clocked(6) -> 720
6! = 720

!!! factorial.__name__ :  clocked !!!


In [22]:
# Example 7-17. An improved clock decorator.
# clockdeco2.py
import time
import functools
def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.time()
        result = func(*args, **kwargs)
        elapsed = time.time() - t0
        name = func.__name__
        arg_lst = []
        if args:
            arg_lst.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
            arg_lst.append(', '.join(pairs))
        arg_str = ', '.join(arg_lst)
        print('[%0.8fs] %s(%s) -> %r ' % (elapsed, name, arg_str, result))
        return result
    return clocked
@clock
def factorial(n):
    """factorial DocString"""
    return 1 if n < 2 else n*factorial(n-1)
print("!!! factorial.__name__ : ", factorial.__name__, "!!!")
print("!!! factorial.__doc__ : ", factorial.__doc__, "!!!")

!!! factorial.__name__ :  factorial !!!
!!! factorial.__doc__ :  factorial DocString !!!


In [24]:
# Example 7-18. The very costly recursive way to compute the Nth number in the Fibonacci series.
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)
if __name__=='__main__':
    print(fibonacci(6))

[0.00000119s] fibonacci(0) -> 0 
[0.00000238s] fibonacci(1) -> 1 
[0.00197339s] fibonacci(2) -> 1 
[0.00000143s] fibonacci(1) -> 1 
[0.00000119s] fibonacci(0) -> 0 
[0.00000167s] fibonacci(1) -> 1 
[0.00098515s] fibonacci(2) -> 1 
[0.00168419s] fibonacci(3) -> 2 
[0.00540257s] fibonacci(4) -> 3 
[0.00000119s] fibonacci(1) -> 1 
[0.00000119s] fibonacci(0) -> 0 
[0.00000167s] fibonacci(1) -> 1 
[0.00067043s] fibonacci(2) -> 1 
[0.00125813s] fibonacci(3) -> 2 
[0.00000072s] fibonacci(0) -> 0 
[0.00000167s] fibonacci(1) -> 1 
[0.00061250s] fibonacci(2) -> 1 
[0.00000095s] fibonacci(1) -> 1 
[0.00000119s] fibonacci(0) -> 0 
[0.00000167s] fibonacci(1) -> 1 
[0.00077438s] fibonacci(2) -> 1 
[0.00161290s] fibonacci(3) -> 2 
[0.00289869s] fibonacci(4) -> 3 
[0.00481629s] fibonacci(5) -> 5 
[0.01090550s] fibonacci(6) -> 8 
8


In [27]:
# Example 7-19. Faster implementation using caching.
import functools
@functools.lru_cache() # 1
@clock # 2
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)
if __name__=='__main__':
    print(fibonacci(6))

[0.00000119s] fibonacci(0) -> 0 
[0.00000215s] fibonacci(1) -> 1 
[0.00159407s] fibonacci(2) -> 1 
[0.00000358s] fibonacci(3) -> 2 
[0.00222468s] fibonacci(4) -> 3 
[0.00000286s] fibonacci(5) -> 5 
[0.00284338s] fibonacci(6) -> 8 
8


In [35]:
# Example 7-20. htmlize generates HTML tailored to different object types
import html
def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)
print("htmlize({1, 2, 3}) : ", htmlize({1, 2, 3}))
print("htmlize(abs) : ", htmlize(abs))
print("htmlize('Heimlich & Co.\\n- a game' : ", htmlize('Heimlich & Co.\n- a game'))
print("htmlize(42) : ", htmlize(42))
print("htmlize(['alpha', 66, {3, 2, 1}]) : \n", htmlize(['alpha', 66, {3, 2, 1}]))

htmlize({1, 2, 3}) :  <pre>{1, 2, 3}</pre>
htmlize(abs) :  <pre>&lt;built-in function abs&gt;</pre>
htmlize('Heimlich & Co.\n- a game' :  <pre>&#x27;Heimlich &amp; Co.\n- a game&#x27;</pre>
htmlize(42) :  <pre>42</pre>
htmlize(['alpha', 66, {3, 2, 1}]) : 
 <pre>[&#x27;alpha&#x27;, 66, {1, 2, 3}]</pre>


In [38]:
# Example 7-21. singledispatch creates a custom htmlize.register to bundle several functions into a generic function.
from functools import singledispatch
from collections import abc
import numbers
import html
@singledispatch # 1
def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)
@htmlize.register(str) # 2
def _(text): # 3
    content = html.escape(text).replace('\n', '<br>\n')
    return '<p>{0}</p>'.format(content)
@htmlize.register(numbers.Integral) # 4
def _(n):
    return '<pre>{0} (0x{0:x})</pre>'.format(n)
@htmlize.register(tuple) # 5
@htmlize.register(abc.MutableSequence)
def _(seq):
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n</ul>'

print("htmlize({1, 2, 3}) : ", htmlize({1, 2, 3}))
print("htmlize(abs) : ", htmlize(abs))
print("htmlize('Heimlich & Co.\\n- a game' : ", htmlize('Heimlich & Co.\n- a game'))
print("htmlize(42) : ", htmlize(42))
print("htmlize(['alpha', 66, {3, 2, 1}]) : \n", htmlize(['alpha', 66, {3, 2, 1}]))

htmlize({1, 2, 3}) :  <pre>{1, 2, 3}</pre>
htmlize(abs) :  <pre>&lt;built-in function abs&gt;</pre>
htmlize('Heimlich & Co.\n- a game' :  <p>Heimlich &amp; Co.<br>
- a game</p>
htmlize(42) :  <pre>42 (0x2a)</pre>
htmlize(['alpha', 66, {3, 2, 1}]) : 
 <ul>
<li><p>alpha</p></li>
<li><pre>66 (0x42)</pre></li>
<li><pre>{1, 2, 3}</pre></li>
</ul>


In [ ]:
# Stacked decorators
@d1
@d2
def f():
    print('f')
    
# Is the same as:
def f():
    print('f')
f = d1(d2(f))

In [1]:
# Example 7-22. Abridged registration.py module from Example 7-2, repeated here for convenience.
registry = []
def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')
    
print('running main()')
print('registry ->', registry)
f1()

running register(<function f1 at 0x7f57c8194158>)
running main()
registry -> [<function f1 at 0x7f57c8194158>]
running f1()


In [9]:
# To accept parameters, the new register decorator must be called as a function.
registry = set()
def register(active=True):
    def decorate(func):
        print('running register(active=%s)->decorate(%s)' % (active, func))
        if active:
            registry.add(func)
        else:
            registry.discard(func)
        return func
    return decorate
@register(active=False)
def f1():
    print('running f1()')
@register()
def f2():
    print('running f2()')
def f3():
    print('running f3()')

print("registry : ", registry ) # When the module is imported, f2 is in the registry.
print()
register()(f3) # !!! the syntax wihout the @
register(active=False)(f2) # This call removes f2 from the registry
print()
print("registry : ", registry )

running register(active=False)->decorate(<function f1 at 0x7f57c81121e0>)
running register(active=True)->decorate(<function f2 at 0x7f57c8107400>)
registry :  {<function f2 at 0x7f57c8107400>}

running register(active=True)->decorate(<function f3 at 0x7f57c8112598>)
running register(active=False)->decorate(<function f2 at 0x7f57c8107400>)

registry :  {<function f3 at 0x7f57c8112598>}


In [34]:
# Example 7-25. Module clockdeco_param.py: the parametrized clock decorator
import time
DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'
def clock(fmt=DEFAULT_FMT):  # <1>
    def decorate(func):      # <2>
        def clocked(*_args): # <3>
            t0 = time.time()
            _result = func(*_args)  # <4>
            elapsed = time.time() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)  # <5>
            result = repr(_result)  # <6>
            print(fmt.format(**locals()))  # <7>
            return _result  # <8>
        return clocked  # <9>
    return decorate  # <10>

if __name__ == '__main__':

    @clock()  # <11>
    def snooze(seconds):
        time.sleep(seconds)

    for i in range(3):
        snooze(.123)

[0.12792087s] snooze(0.123) -> None
[0.12495661s] snooze(0.123) -> None
[0.12414408s] snooze(0.123) -> None


In [35]:
# Example 7-26. clockdeco_param_demo1.py
@clock('{name}: {elapsed}s')
def snooze(seconds):
    time.sleep(seconds)
for i in range(3):
    snooze(.123)

snooze: 0.12451910972595215s
snooze: 0.1244819164276123s
snooze: 0.12383413314819336s


In [36]:
# Example 7-27. clockdeco_param_demo2.py
@clock('{name}({args}) dt={elapsed:0.3f}s')
def snooze(seconds):
    time.sleep(seconds)
for i in range(3):
    snooze(.123)

snooze(0.123) dt=0.125s
snooze(0.123) dt=0.123s
snooze(0.123) dt=0.124s
